In [ ]:
arrow::read_parquet(here("data", "atpTrans.parquet")) -> atp
atp %>% glimpse()

Rows: 682,069
Columns: 27
$ winner.name            <chr> "Patrick Rafter", "Dennis Van Scheppingen", "Ja…
$ winner.id              <chr> "r255", "s559", "s381", "d214", "r255", "s559",…
$ winner.rank            <int> 6, 126, 26, 85, 6, 126, 26, 11, 85, 6, 12, 55, …
$ winner.bornAt          <chr> "Australia", "Netherlands", "Netherlands", "Cze…
$ winner.domHand         <chr> "Right-Handed", "Right-Handed", "Left-Handed", …
$ winner.backHand        <chr> "One-Handed Backhand", "Unknown Backhand", "Unk…
$ winner.height          <int> 185, 185, 183, 188, 185, 185, 183, 185, 188, 18…
$ looser.name            <chr> "Martin Damm", "Richard Krajicek", "Steve Campb…
$ looser.id              <chr> "d214", "k214", "c355", "k237", "s572", "w254",…
$ looser.rank            <int> 85, 12, 95, 11, 55, 189, 58, 60, 100, 94, 42, 3…
$ looser.bornAt          <chr> "Czech Republic", "Netherlands", "USA", "Czech …
$ looser.domHand         <chr> "Right-Handed", "Right-Handed", "Right-Handed",…
$ looser.backH

In [ ]:
atp %>% filter(tournament.location == "Australia") -> atpA

In [ ]:
# parquet doesn't seem to save factor
atpA %>% mutate(
        winner.backHand = winner.backHand             %>% as.factor,
        winner.domHand  = winner.domHand              %>% as.factor,
        looser.backHand = looser.backHand             %>% as.factor,
        looser.domHand  = looser.domHand              %>% as.factor,
        match.tournamentRound = match.tournamentRound %>% as.factor,
        tournament.ground = tournament.ground         %>% as.factor,
        tournament.location = tournament.location     %>% as.factor
    ) -> atpAF

In [ ]:
if (FALSE) {
    atpAF %>% SmartEDA::ExpReport(op_file = "temp.html")
}

In [ ]:
(atpAF %>% nrow() -> nmatches.inicial)

[1] 21388

In [ ]:
# things to fix:
# prize ✔
# maybe dont include those that are both heights missing✔
# remove rets walkovers and defs✔
# decide on how to remove best of 5 tournaments✔
# turn nas into a factor too in backhand and domHand (backhand already has an unkown)
# fazer eda do rank para perceber como tratalos
# tratar heights✔

In [ ]:
# prize
atpAF %>% 
    select(tournament.prize) %>%
    filter(str_detect(tournament.prize, fixed("$"), negate=T))

tournament.prize
<chr>


In [ ]:
# no prizes without dollar
atpAF %>%
    mutate(tp = str_remove_all(tournament.prize, r"{[,\$]}")) %>%
    filter(str_detect(tp, r"{[^0-9]}")) %>% 
    select(tp)

tp
<chr>
A9609870
A9609870
A9609870
A9609870
A9609870
A9609870
A9609870
A9609870
A9609870


In [ ]:
# australian dollar
atpAF %>%
    mutate(tp = str_remove_all(tournament.prize, r"{[,\$A]}")) %>%
    filter(str_detect(tp, r"{[^0-9]}")) %>% 
    select(tp)

tp
<chr>


In [ ]:
doll_to_aus <- \(x) x*1.48
atpAF %>%
    rename(tp = tournament.prize) %>% 
    mutate(tournament.prize = case_when(
        tp %>% is.na()                 ~ NA,
        tp %>% str_detect(fixed("A$")) ~ tp %>% str_remove_all(r"{[\A$,]}") %>% as.double(),
        tp %>% str_detect(fixed("$"))  ~ tp %>% str_remove_all(r"{[\$,]}")  %>% as.double() %>% doll_to_aus(),
        TRUE ~ 9e+9
     )) %>% select(-tp) -> atpAFP
atpAFP %>% filter(tournament.prize == 9e+9) %>% nrow()

Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `tournament.prize = case_when(...)`.
Caused by warning in `doll_to_aus()`:
! NAs introduced by coercion"


[1] 0

In [ ]:
library(skimr)
# atpAFP %>% skim(winner.height, looser.height) # yields error because of jup

In [ ]:
atpAFP %>% mutate(
    winner.height = ifelse(winner.height == 0, NA, winner.height), 
    looser.height = ifelse(looser.height == 0, NA, looser.height), 
) -> atpAFPH.wNA
# atpAFPH.wNA %>% skim(winner.height, looser.height)
# interpolate heights?
# maybe if we don't get enough observations

In [ ]:
atpAFPH <- atpAFPH.wNA %>% filter(!is.na(winner.height) & !is.na(looser.height))
# atpAFPH %>% skim(winner.height, looser.height)

── Data Summary ────────────────────────
                           Values    
Name                       Piped data
Number of rows             14357     
Number of columns          27        
_______________________              
Column type frequency:               
  numeric                  2         
________________________             
Group variables            None      

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate mean   sd p0 p25 p50 p75 p100 hist 
1 winner.height         0             1 185. 11.3 15 180 185 188  211 ▁▁▁▁▇
2 looser.height         0             1 184. 10.7 15 180 185 188  211 ▁▁▁▁▇


ERROR: Error in is.null(text_repr) || nchar(text_repr) == 0L: 'length = 12' in coercion to 'logical(1)'


In [ ]:
atpAFPH %>% filter(winner.height < 160) %>% distinct(winner.id)

winner.id
<chr>
s406
h431


In [ ]:
atpAFPH %>% filter(looser.height < 160) %>% distinct(looser.id)

looser.id
<chr>
s406
h431


TODO texto mas

* h431 -> 180 cm
* s406 -> 188 cm

In [ ]:
atpAFPHF <- atpAFPH %>% mutate(winner.height = case_when(
    winner.id == "h431" ~ 180,
    winner.id == "s406" ~ 188,
    TRUE ~ winner.height
), looser.height = case_when(
    looser.id == "h431" ~ 180,
    looser.id == "s406" ~ 188,
    TRUE ~ looser.height
))
# atpAFPHF %>% skim(winner.height, looser.height)

In [ ]:
atpAFPHFR <- atpAFPHF %>% 
    filter(!match.wasRet, !match.wasWalkover, !match.wasDef) %>%
    select(-match.wasRet, -match.wasWalkover, -match.wasDef)
# atpAFPHFR %>% select(starts_with("match")) %>% skim()

In [ ]:
# tournaments best of 5
atpAFPHFR -> atp.wbo5
atp.wbo5 %>% 
    group_by(tournament.name) %>% 
    filter(!any(match.setsCount > 3)) %>%
    ungroup() -> atp.wo45

In [ ]:
atp.wo45 %>% 
    group_by(tournament.name) %>% 
    filter(all(match.setsCount == 3)) %>% 
    summarise()

tournament.name
<chr>
AUS v. MAR WG PO
AUS v. SWE WG 1st RD
AUS vs. SVK WG Play-Off
Adelaide-2
ROM V AUS QF


TODO ver quais destes sao melhor de 3 e melhores de 5, dps tirar esses

In [ ]:
toRem = c()
atp.wo45 %>% filter(!(tournament.name %in% toRem)) -> atp.wobo5

In [ ]:
# features q faltam:
# diferenca de alturas
# media de alturas
# idades (*)
# numero de jogos jogados - diferenca/numero de wins/winrate
# frequencia de jogos do pais de naturalidade
# se pais tem jogos - condicao/contagem/diferenca/diferenca de condicao
# domhand condicao
# backhand condicao
# se e atp
# frequencia do torneio
# mes do torneio
# distancia ao ultimo torneio/primeiro
# premio
# distancia da ronda final
# diferenca de rank /categoria de rank
# maybe join domHand